In [37]:
import requests
from bs4 import BeautifulSoup
import re

def get_details(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    try:
        title = soup.find('div', class_='page-header').h1.text.strip()
    except:
        title = None

    try:
        author = soup.find('p', class_='lead author-list pull-left').text.strip()
    except:
        author = None

    try:
        date = soup.find('p', class_='lead pub_type pull-right').text.strip()
    except:
        date = None

    table = soup.find('table', class_='table')
    try:
        supervisor_label = table.find('span', string=re.compile('Thesis Advisor'))
        supervisor = supervisor_label.parent.parent.findAll('td')[1].text.strip()
        supervisor = supervisor.split('\n')
        supervisor = [s.replace(',', '') for s in supervisor]
    except:
        supervisor = None

    
    try:
        faculty_label = table.find('td', string=re.compile('Organisation'))
        faculty = faculty_label.findNext('td').text.strip()
    except:
        faculty = None

    try:
        abstract = soup.find('p', class_='abstract').text.strip()
        
    except:
        abstract = None

    try:
        keywords_label = table.find('span', string=re.compile('Keyword'))
        keywords = keywords_label.parent.parent.findAll('td')[1].text.strip()
        keywords = keywords.split(',')
        keywords = [k.strip() for k in keywords]
    except:
        keywords = None


    #make dict
    details = {
        'title': title,
        'author': author,
        'date': date,
        'supervisor': supervisor,
        'faculty': faculty,
        'abstract': abstract,
        'keywords': keywords
    }

    return details

In [38]:
#get ids from test.txt
links = []
with open('links.txt', 'r') as f:
    for line in f:
        links.append(line.strip())

len(links)

10793

In [39]:
from tqdm.auto import tqdm
import pandas as pd

#divide the links in chunks of 500
chunks = [links[x:x+500] for x in range(0, len(links), 500)]

#loop over chunks, make a csv for each
for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    #check if theses_i.csv already exists, if so, skip
    try:
        pd.read_csv('data/theses_' + str(i) + '.csv')
        continue
    except:
        pass

    theses = pd.DataFrame(columns=['title', 'author', 'date', 'supervisor', 'faculty', 'abstract', 'keywords']) 

    for link in tqdm(chunk):
        thesis = get_details(link)
        theses = theses._append(thesis, ignore_index=True)

    theses.to_csv('data/theses_' + str(i) + '.csv', index=False)




  0%|          | 0/22 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

...

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/293 [00:00<?, ?it/s]

In [40]:
df_merged = pd.DataFrame(columns=['title', 'author', 'date', 'supervisor', 'faculty', 'abstract', 'keywords'])
for i in tqdm(range(len(chunks))):
    df = pd.read_csv('data/theses_' + str(i) + '.csv')
    df_merged = df_merged._append(df, ignore_index=True)

df_merged.to_csv('theses.csv', index=False)

  0%|          | 0/22 [00:00<?, ?it/s]